# Preliminary tenure choice model 4/8/2018

Paul Waddell | Python 3.6

Based on the urbansim_templates project led by Sam Maurer

In [1]:
import os; os.chdir('../')

In [2]:
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import BinaryLogitStep
import orca

/Users/Paul/anaconda3/envs/ual-model/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
# Load any script-based Orca registrations
from scripts import datasources
from scripts import models

In [4]:
orca.list_tables()

['households', 'buildings']

In [5]:
for table_name in orca.list_tables():
    print(table_name.upper())
    print(orca.get_table(table_name).to_frame().columns.tolist())
    print()

HOUSEHOLDS
['Unnamed: 0', 'serialno', 'persons', 'building_type', 'cars', 'income', 'race_of_head', 'hispanic_head', 'age_of_head', 'workers', 'state', 'county', 'tract', 'block group', 'children', 'tenure', 'recent_mover']

BUILDINGS
['building_id', 'parcel_id', 'development_type_id', 'improvement_value', 'residential_units', 'residential_sqft', 'sqft_per_unit', 'non_residential_sqft', 'building_sqft', 'nonres_rent_per_sqft', 'res_price_per_sqft', 'stories', 'year_built', 'redfin_sale_price', 'redfin_sale_year', 'redfin_home_type', 'costar_property_type', 'costar_rent', 'building_type_id']



### Turn 'tenure' into a 0/1 binary variable

In [46]:
h = orca.get_table('households').to_frame()

In [52]:
h.describe()

,Unnamed: 0,serialno,persons,building_type,cars,income,race_of_head,age_of_head,workers,state,county,tract,block group,children,tenure,recent_mover
count,2.518676e+06,2.518676e+06,2.518676e+06,2.518676e+06,2.518676e+06,2.518676e+06,2.518676e+06,2.518676e+06,1.980656e+06,2518676.0,2.518676e+06,2.518676e+06,2.518676e+06,800349.000000,2.518676e+06,2.518676e+06
mean,1.259338e+06,2.011050e+12,2.648932e+00,3.640697e+00,1.876059e+00,1.148300e+05,2.905535e+00,5.281594e+01,1.666675e+00,6.0,5.162579e+01,3.667226e+05,2.124913e+00,1.782495,5.709587e-01,3.952354e-02
std,7.270793e+05,1.415768e+09,1.580338e+00,2.357789e+00,1.143827e+00,1.089306e+05,2.677256e+00,1.564460e+01,8.231937e-01,0.0,3.716657e+01,1.829696e+05,1.099937e+00,0.900753,4.949393e-01,1.948370e-01
min,0.000000e+00,2.009000e+12,1.000000e+00,1.000000e+00,0.000000e+00,-1.160000e+04,1.000000e+00,1.500000e+01,1.000000e+00,6.0,1.000000e+00,1.010000e+04,0.000000e+00,1.000000,0.000000e+00,0.000000e+00
25%,6.296688e+05,2.010000e+12,1.000000e+00,2.000000e+00,1.000000e+00,4.110000e+04,1.000000e+00,4.100000e+01,1.000000e+00,6.0,1.300000e+01,2.527060e+05,1.000000e+00,1.000000,0.000000e+00,0.000000e+00
50%,1.259338e+06,2.011001e+12,2.000000e+00,2.000000e+00,2.000000e+00,8.550000e+04,1.000000e+00,5.200000e+01,2.000000e+00,6.0,7.500000e+01,4.231000e+05,2.000000e+00,2.000000,1.000000e+00,0.000000e+00
75%,1.889006e+06,2.012001e+12,4.000000e+00,5.000000e+00,2.000000e+00,1.522500e+05,6.000000e+00,6.300000e+01,2.000000e+00,6.0,8.500000e+01,5.051000e+05,3.000000e+00,2.000000,1.000000e+00,0.000000e+00
max,2.518675e+06,2.013001e+12,2.000000e+01,1.000000e+01,6.000000e+00,1.397000e+06,9.000000e+00,9.400000e+01,1.300000e+01,6.0,9.700000e+01,9.901000e+05,7.000000e+00,9.000000,1.000000e+00,1.000000e+00


In [47]:
h.tenure.value_counts()

1    1438060
0    1080616
Name: tenure, dtype: int64

In [48]:
h.loc[h.tenure==2, 'tenure'] = 0
h.tenure.value_counts()

1    1438060
0    1080616
Name: tenure, dtype: int64

In [49]:
# Update column
orca.get_table('households').update_col_from_series('tenure', h.tenure)

# Check that it worked
orca.get_table('households').to_frame().tenure.value_counts()

1    1438060
0    1080616
Name: tenure, dtype: int64

### Estimate a model

In [53]:
m = BinaryLogitStep()
m.tables = (['households'])
m.model_expression = 'tenure ~ np.log(income+1) + workers + cars + age_of_head + building_type==2 + race_of_head==2 +\
hispanic_head + recent_mover'
m.filters = ['income < 500000']

In [54]:
m.fit()

<string>:1: RuntimeWarning: invalid value encountered in log
<string>:1: RuntimeWarning: invalid value encountered in log


Optimization terminated successfully.
         Current function value: 0.438876
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                 tenure   No. Observations:              1951643
Model:                          Logit   Df Residuals:                  1951634
Method:                           MLE   Df Model:                            8
Date:                Sun, 08 Apr 2018   Pseudo R-squ.:                  0.3585
Time:                        20:56:56   Log-Likelihood:            -8.5653e+05
converged:                       True   LL-Null:                   -1.3352e+06
                                        LLR p-value:                     0.000
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                    -14.4507      0.034   -423.296      0.000     -